In [16]:
import logging
import pandas as pd
import os
import json
import time
import requests

In [17]:
with open("session.txt", "r") as f:
    sessionid = f.read()

In [18]:
headers_for_html_fetching = {
            'Authority': 'www.threads.net',
            'Accept': (
                'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;'
                'q=0.8,application/signed-exchange;v=b3;q=0.7'
            ),
            'Accept-Language': 'en-US,en;q=0.9',
            'Cache-Control': 'no-cache',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Origin': 'https://www.threads.net',
            'Pragma': 'no-cache',
            'Referer': 'https://www.instagram.com',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'cross-site',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': (
                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) '
                'AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1.2 Safari/605.1.15'
            ),
        }

In [19]:
import re
def get_threads_userpk(username:str):
    cookies = {
            "sessionid": sessionid
    }
    response = requests.get(
                url=f'https://www.threads.net/@{username}',
                headers=headers_for_html_fetching,
                cookies=cookies
            )
    try:
        if "viewer does not follow private user" in response.text:
            return None
        text = response.text
        user_id_key_value = re.search('"user_id":"(\\d+)",', text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

get_threads_userpk("shelli1370")

'63074975810'

In [20]:
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery(sessionid=sessionid)
# t.retrieve_follower_by_userid_after("69655371289","0")

In [21]:

def get_follower_data(data: list):
    try:
        return [
            {
                "pk": node.get("pk", ""),
                "username": node.get("username", ""),
                "follower_count": node.get("follower_count", "")
            }
            for d in data
            if (node := d.get("node", {}))
        ]
    except Exception as e:
        raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")


progress_tracker = {}
filejson = "progress2.json"

def save_progress(userid: str, batch_number: int, results: list):
    """Salva il batch_number corrente e i risultati ottenuti per un determinato userid."""
    progress_tracker[userid] = {"batch_number": batch_number, "results": results}
    with open(filejson, "w") as f:
        json.dump(progress_tracker, f)

def load_progress(userid: str):
    """Carica il batch_number e i risultati salvati per un determinato userid."""
    try:
        with open(filejson, "r") as f:
            data = json.load(f)
            return data.get(userid, {"batch_number": 1, "results": []})
    except (FileNotFoundError, json.JSONDecodeError):
        return {"batch_number": 1, "results": []}
def remove_progress(userid: str):
    """Rimuove il progresso salvato per un determinato userid."""
    try:
        with open(filejson, "r") as f:
            data = json.load(f)
        
        if userid in data:
            del data[userid]
            with open(filejson, "w") as f:
                json.dump(data, f)
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
def get_all_follower(userid: str):
    progress = load_progress(userid)
    batch_number = progress["batch_number"]
    results = progress["results"]
    allpk = [res["pk"] for res in results]
    try:
        has_next_page = True
        while has_next_page:
            try:
                print("Pagina:", batch_number)
                response = t.retrieve_follower_by_userid_after(userid, str(25 * batch_number-1))
                batch_number += 1
                data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
                if data.get("edges", []) == []:
                    return [{
                        "pk": userid,
                        "username": None,
                        "follower_count": None
                    }]
                d_out = get_follower_data(data.get("edges", []))
                for res in d_out:
                    if res["pk"] in allpk:
                        remove_progress(userid)
                        return results
                    allpk.append(res["pk"])
                results.extend(d_out)
                
                has_next_page = data.get("page_info", {}).get("has_next_page", False)
                
                # Salva il progresso dopo ogni batch
                save_progress(userid, batch_number, results)
                
            except Exception as e:
                save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore
                raise RuntimeError(f"Errore nel recupero della paginazione dei follower: {e}")
    
    except Exception as e:
        save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore generale
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    remove_progress(userid)
    return results

In [22]:
df = pd.read_csv("followers_ChatGPT.csv")
df['follower_pk'] = df['follower_pk'].astype(str)
unique_user_pk = df['follower_pk'].unique()
len(unique_user_pk)

1950

In [23]:
dff = pd.read_csv("followers_ChatGPT.csv")
dff.head(1)

,user_pk,follower_pk,follower_username,follower_count
0,31805920812,68037739428,_rafif1734_,0


In [24]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "ffGPT1.csv"
processed_users = load_processed_users(csv_file)

In [25]:
len(processed_users)

0

In [26]:
print(len(unique_user_pk))
unique_user_pk = unique_user_pk[unique_user_pk != "339473819"]

print(len(unique_user_pk))

1950
1950


In [27]:
def fetch_followers(pk,user_pk):
    print(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": user_pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"],
                "user_threads_userpk": pk
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None

follower_list = []
batch_size = 1


print("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        print(f"Skipping already processed user_pk: {user_pk}")
        continue
    processed_users.update([user_pk])
    username= dff[dff["follower_pk"] == int(user_pk)]["follower_username"].unique()[0]
    print(user_pk, username)
    threads_user_pk = get_threads_userpk(str(username))
    if threads_user_pk is None:
        follower_list.extend([{
                "user_pk": user_pk,
                "follower_pk": None,
                "follower_username": None,
                "follower_count": None,
                "user_threads_userpk": None
        }])
        continue
    results = fetch_followers(threads_user_pk,user_pk)
    if results is None:
        break
    follower_list.extend(results)
    
    if follower_list:
        print(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []

if follower_list:
    print(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)   
    save_to_csv(follower_df, csv_file)

print("Follower data collection completed")


Starting follower data collection
68037739428 _rafif1734_
70726149166 arts.wh0re
Fetching followers for user_pk: 71347280359
Pagina: 1
Pagina: 2
Processing batch of 25 followers
49174626074 ganaa.gansuhk.52
60882225687 cc.jayyyy
Fetching followers for user_pk: 65191160590
Pagina: 1
Processing batch of 11 followers
55386498420 aliquashady999


KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv(csv_file)
df_count= df1.drop_duplicates().groupby("user_pk").count()
print(df_count.shape)
df_count

(1, 4)


,follower_pk,follower_username,follower_count,user_threads_userpk
user_pk,,,,
4529033111,49,49,49,49
